In [13]:
# Environment using Python 3.8.11
%pip install opencv-python==4.5.2.52 opencv-contrib-python==4.5.2.52 # Install compatible OpenCV version with WeChatQRCode
%pip install pillow==9.1.0 torch==1.10.2 torchvision==0.11.3
%pip install tqdm==4.64.1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
import gc
from tqdm import tqdm
import cv2
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import PIL.Image as Image

In [2]:
# Choose device for model running, the reconstruction is performed on the CPU by default
# This can be changed by simply replacing .cpu() with .to(device)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Upscaling image using SRCNN model

#### Build architecture

In [3]:
class SuperResolution(nn.Module):
    
    """
    SRCNN Network Architecture as per specified in the paper. 
    The chosen configuration for successive filter sizes are 9-5-5
    The chosed configuration for successive filter depth are 128-64(-3)
    """
    def __init__(self, sub_image: int = 33, spatial: list = [9, 5, 5], filter: list = [128, 64], num_channels: int = 3):
        super().__init__()
        self.layer_1 = nn.Conv2d(num_channels, filter[0], spatial[0], padding = spatial[0] // 2)
        self.layer_2 = nn.Conv2d(filter[0], filter[1], spatial[1], padding = spatial[1] // 2)
        self.layer_3 = nn.Conv2d(filter[1], num_channels, spatial[2], padding = spatial[2] // 2)
        self.relu = nn.ReLU()

    def forward(self, image_batch):
        x = self.layer_1(image_batch)
        x = self.relu(x)
        x = self.layer_2(x)
        y = self.relu(x)
        x = self.layer_3(y)
        return x, y 

In [4]:
def execute(image_in, model, fs = 33, scale = None):
  
    """
    Executes the model trained on colab, on any image given (link or local), with an 
    upscaling factor as mentioned in the arguments. For best results, use a scale of
    2 or lesser, since the model was trained on a scale of 2
    Inputs : image_in               -> torch.tensor representing the image, can be easily obtained from 
                                       transform_image function in this script (torch.tensor)
             model                  -> The trained model, trained using the same patch size 
                                       (object of the model class, inherited from nn.Module) 
             fs                     -> Patch size, on which the model is run (int)
             scale                  -> Scale on which the image is upscaled (float) 
    Outputs: reconstructed_image    -> The higher definition image as output (torch.tensor)
    """
    # Write the transforms and prepare the empty array for the image to be written
    c, h, w = image_in.shape
    scale_transform = transforms.Resize((int(h * scale), int(w * scale)), interpolation=transforms.InterpolationMode.BICUBIC)

    to_pil = transforms.ToPILImage()
    to_tensor = transforms.ToTensor()
    image = to_tensor(scale_transform(to_pil(image_in)))
    n = 0
    c, h, w = image.shape
    image = image.unsqueeze(0)
    image = image.to(device)
    reconstructed_image = torch.zeros_like(image).cpu()
    reconstructed_image_weights = torch.zeros_like(image).cpu()

    # Loop for non overlapping image reconstruction 
    # since overlapping reconstruction needs too much memory even for small images 
    for i in range(h // fs):
      for j in range(w // fs):

        # Clean up memory and track iterations
        gc.collect()
        n += 1

        # Get the j'th (fs, fs) shaped patch of the (i * fs)'th row, 
        # Upscale this patch and write to the empty array at appropriate location  
        patch = image[:, :, i * fs: i * fs + fs, j * fs: j * fs + fs]
        reconstructed_image[:, :, i * fs: i * fs + fs, j * fs: j * fs + fs] = model(patch)[0].cpu().clamp(0, 1)
        reconstructed_image_weights[:, :, i * fs: i * fs + fs, j * fs: j * fs + fs] += torch.ones(1, c, fs, fs)
        
        # This leaves the right and bottom edge black, if the width and height are not divisible by fs
        # Those edge cases are dealt with here
        if j == w // fs - 1:
            patch = image[:, :, i * fs: i * fs + fs, w - fs: w]
            reconstructed_image[:, :, i * fs: i * fs + fs, w - fs: w] = model(patch)[0].cpu().clamp(0, 1)
        if i == h // fs - 1:
            patch = image[:, :, h - fs: h, j * fs: j * fs + fs]
            reconstructed_image[:, :, h - fs: h, j * fs: j * fs + fs] = model(patch)[0].cpu().clamp(0, 1)
          
      # Make the right bottom patch, since none of the edge cases have covered it
      patch = image[:, :, h - fs: h, w - fs: w]
      reconstructed_image[:, :, h - fs: h, w - fs: w] = model(patch)[0].cpu().clamp(0, 1)
    
    return reconstructed_image

#### Upscaled output is obtained and converted to OpenCV Matrix

In [5]:
def upscale(path_to_image, path_to_model):

    # Instantiate model and load state dict using .pth file 
    model = SuperResolution()
    if torch.cuda.is_available():
        model.load_state_dict(torch.load(path_to_model))
    else:
        model.load_state_dict(torch.load(path_to_model, map_location={'cuda:0': 'cpu'}))
    model.to(device)
    model.eval()

    # Run the progressive scan to increase resolution of the image 
    image = Image.open(path_to_image)
    trans = transforms.Compose([transforms.ToTensor()])
    transformed = trans(image)
    reconstructed = execute(transformed, model, scale = 2)  
    to_pil = transforms.ToPILImage()

    # Convert upscaled image to PIL format
    pil_upscaled = to_pil(reconstructed.squeeze())

    # Convert PIL to OpenCV Matrix
    opencv_upscaled = np.array(pil_upscaled) 
    # Convert RGB to BGR 
    opencv_upscaled = opencv_upscaled[:, :, ::-1].copy()

    return opencv_upscaled 

## Automating brightness and contrast enhancement

In [6]:
def convertScale(img, alpha, beta):
    # Add bias and gain to an image with saturation arithmetics. Unlike
    # cv2.convertScaleAbs, it does not take an absolute value, which would lead to
    # nonsensical results (e.g., a pixel at 44 with alpha = 3 and beta = -210
    # becomes 78 with OpenCV, when in fact it should become 0).

    new_img = img * alpha + beta
    new_img[new_img < 0] = 0
    new_img[new_img > 255] = 255
    return new_img.astype(np.uint8)

# Automatic brightness and contrast optimization with optional histogram clipping
def autoBrightnessAndContrast(image, clip_hist_percent):

    # Calculate grayscale histogram
    hist = cv2.calcHist([image],[0],None,[256],[0,256])
    hist_size = len(hist)

    # Calculate cumulative distribution from the histogram
    accumulator = []
    accumulator.append(float(hist[0]))
    for index in range(1, hist_size):
        accumulator.append(accumulator[index -1] + float(hist[index]))

    # Locate points to clip
    maximum = accumulator[-1]
    clip_hist_percent *= (maximum/100.0)
    clip_hist_percent /= 2.0

    # Locate left cut
    minimum_gray = 0
    while accumulator[minimum_gray] < clip_hist_percent:
        minimum_gray += 1

    # Locate right cut
    maximum_gray = hist_size -1
    while accumulator[maximum_gray] >= (maximum - clip_hist_percent):
        maximum_gray -= 1

    # Calculate alpha and beta values
    alpha = 255 / (maximum_gray - minimum_gray)
    beta = -minimum_gray * alpha

    auto_result = convertScale(image, alpha=alpha, beta=beta)
    return auto_result

## Raw input image is ready to be enhanced and decoded

In [17]:
def QRreader(path_to_image, path_to_model, imgsz):

    # declare QR result variable
    qrData = None
    
    # enhance image to super resolution
    upscaled_image = upscale(path_to_image, path_to_model)

    # gray scale for faster computation
    gray = cv2.cvtColor(upscaled_image, cv2.COLOR_BGR2GRAY)

    if gray is not None:

        # stop program if no QR code detected
        try:
            h, w = gray.shape[:2]  
            
            # resize for faster decoding with optional value
            ratio = imgsz / w
            resize = cv2.resize(gray, (imgsz, round(h * ratio)), interpolation = cv2.INTER_AREA)

            # deblur
            gaussian = cv2.GaussianBlur(resize, (15, 15), 10.0)
            unsharp = cv2.addWeighted(resize, 8, gaussian, -7, 0)

            # automatic brightness and contrast optimization with optional histogram clipping
            brightness_contrast = autoBrightnessAndContrast(unsharp, 5)

            # binarization
            _, bin = cv2.threshold(brightness_contrast, 0, 255, cv2.THRESH_OTSU)
            
            # decode using OpenCV WechatQRCode
            detector = cv2.wechat_qrcode_WeChatQRCode("./model/detect.prototxt",
                                                      "./model/detect.caffemodel",
                                                      "./model/sr.prototxt",
                                                      "./model/sr.caffemodel")

            result = detector.detectAndDecode(bin)

            if len(result[0]) != 0:
                qrData = result[0][0]        
        except:
            qrData = None
        
    return qrData, bin

In [23]:
# Declare QR images and trained model directory
sourcePath = "qr_large"
superResModelPath = "SRCNN/isr_best.pth"
# Resize image to 320 x 320
# recommended value in range (250, 350)
imgsz = 280

# Count number of images scanned
imgNum = 0 
decodeSuccess = 0
# Decode a series of QR code images in "sourceFolder"
for f in tqdm(os.listdir(sourcePath), desc = "Decoding in progess "):
    try:
        imgPath = os.path.join(sourcePath, f)
        result, img = QRreader(imgPath, superResModelPath, imgsz)
        
        if result is not None:
            # print(f"{result}\n")
            decodeSuccess += 1
        imgNum += 1
    except:
        raise AssertionError

rate = decodeSuccess / imgNum * 100
print(f"DECODE SUCCESSFULLY: {decodeSuccess}/{imgNum}\nRATE: {rate:.2f}%")

Decoding in progess :  16%|█▋        | 500/3039 [18:44<1:35:10,  2.25s/it]

DECODE SUCCESSFULLY: 252/501
RATE: 50.30%
